# KGOntologies Example N#1
## AcmeBank Stafflist

Load a data file containing members of staff and the departments in which they are employed and process it into a marked-up knowledge-graph for visualisation. 



In [8]:
import os, sys

srcdir=r'..'
if srcdir not in sys.path:
    sys.path.append(srcdir)

from IPython.display import display, Image, HTML
import pandas as pd
import src.graphvisutils_gravis as graphvisutils_gravis 
import src.graphloader as graphloader
import src.queryaugment as queryaugment

import src.visgui as gui

import gravis as gv

import rdflib
from rdflib import URIRef, Literal, Namespace
import uuid
from hashlib import md5
import json

from functools import partial

In [2]:
# Locate and read the data file
data_df = pd.read_excel("../data/AcmeBankStaffList.xlsx", na_values='None')

In [3]:
# Load and bind associated ontologies for additional data enrichment
kgdmcar_g = rdflib.Graph()
kgdmcar_g.parse ('../ontologies/kgdmcar.owl', format='xml')

kgnaming_g = rdflib.Graph()
kgnaming_g.parse ('../ontologies/kgnaming.owl', format='xml')

ontology_graph = rdflib.Graph(bind_namespaces="rdflib")
for t in kgnaming_g.triples((None, None, None)):
    ontology_graph.add(t)

for t in kgdmcar_g.triples((None, None, None)):
    ontology_graph.add(t)

In [5]:
# Convert the data into a raw graph format
rawg=graphloader.rdflib_graph_from_dataframe(data_df[0:500])

# Open a serialisation config file and convert the raw graph into triples describing entities, relations and literals
serial_config = json.load(open("../serialisations/acmestaff_serialisation.json", "r"))
load_graph  = graphloader.process_anonymous_data_graph(rawg, serial_config, data_namespace="http://data#")
# Bind some appropriate namespaces to the resulting subgraph object
namespace_dict = {  "kgnam" : Namespace("http://www.semanticweb.org/tomk/ontologies/2025/5/kgnaming#"),
                    "dmcar" : Namespace("http://www.semanticweb.org/tomk/ontologies/2025/5/kgdmcar#"),
                    "acme" : Namespace("http://www.acmebank.org/")}
graphloader.bind_namespaces(load_graph, namespace_dict)


In [9]:

test_ui = gui.gui_visualisation_control(load_graph)
test_ui.control

set()
set()
{rdflib.term.URIRef('http://www.acmebank.org/Department'): {'id': rdflib.term.URIRef('http://www.acmebank.org/Department'), 'name': 'acme:Department', 'shape': 'circle', 'size': 20, 'color': '#001219', 'selected': True, 'label_size': 20}, rdflib.term.URIRef('http://www.acmebank.org/Person'): {'id': rdflib.term.URIRef('http://www.acmebank.org/Person'), 'name': 'acme:Person', 'shape': 'circle', 'size': 20, 'color': '#005f73', 'selected': True, 'label_size': 20}, rdflib.term.URIRef('http://www.acmebank.org/Role'): {'id': rdflib.term.URIRef('http://www.acmebank.org/Role'), 'name': 'acme:Role', 'shape': 'circle', 'size': 20, 'color': '#0a9396', 'selected': True, 'label_size': 20}}


In [ ]:
gv.vis(load_nx_g, **vis_graph_settings)

In [ ]:
# Run a SPARQL Query to Retrieve only Active Staff Members (i.e. those with an assigned role)
# Note that this query is only returning nodes that meed certain conditions, the resulting
# resultset is not in s,p,o form, just contains a collection of node-uris.

active_staff_q = """
PREFIX acme: <http://www.acmebank.org/>

SELECT ?s ?q
WHERE 
    {
    
    {SELECT ?s ?q
    WHERE { 
    ?s a ?type.
    ?s acme:HasRole ?q.
    ?q a acme:Role.
    VALUES ?type {acme:Person} .
    }}

UNION

    {SELECT ?s
    WHERE { 
    ?s a ?type.
    VALUES ?type {acme:Role} .
    }}

UNION
    
    {SELECT ?s
    WHERE { 
    ?s a ?type.
    VALUES ?type {acme:Department} .
    }}

}


"""


In [ ]:
# Run the query, then augment it with the get_x options, this allows for simpler workflow and easier sparql queries to be performed
# Augmentation results in a new subgraph, which is driven from the parameters
subgraph=queryaugment.augment_sparql_to_graph(load_graph, 
                          active_staff_q, 
                          get_types=True, 
                          get_labels=True, 
                          get_literals=True, 
                          get_cross_links=True, 
                          sparql_option=False)

In [ ]:
# Bind some appropriate namespaces to the resulting subgraph object
KGNAM = Namespace("http://www.semanticweb.org/tomk/ontologies/2025/5/kgnaming#")
DMCAR = Namespace("http://www.semanticweb.org/tomk/ontologies/2025/5/kgdmcar#")
ACME = Namespace("http://www.acmebank.org/")
subgraph.bind("kgnam", KGNAM)
subgraph.bind("dmcar", DMCAR)
subgraph.bind("acme", ACME)

In [ ]:
# Generate a networkx "property-graph" representation of the knowledge=graph for passing to the visualiser
nx_g = graphvisutils_gravis.rdflib_graph_to_networkx_for_gravis(subgraph, 
                                                                ontology_graph, 
                                                                hide_types=True, 
                                                                hide_literals=True, 
                                                                node_colour_scheme="bold4")

In [ ]:
# Set default visualiser settings
# Layout algorithm options:
#    “barnesHut”, “forceAtlas2Based”, “repulsion”, “hierarchicalRepulsion”
vis_graph_settings = {
    "node_label_data_source" : "label", 
    "node_size_data_source" : "size", 
    "node_label_size_factor" : 1.5, 
    "node_size_factor" : 2.5, 
    "edge_size_data_source" : "size", 
    "edge_label_data_source" : "label", 
    "edge_size_factor" : 2.5, 
    "show_edge_label" : True,
    "edge_curvature" : 0.2,
    "graph_height" : 800,
    "details_height" : 300, 
    "layout_algorithm" : "forceAtlas2Based", 
    #"layout_algorithm" : "hierarchicalRepulsion", 
    #"gravitational_constant" : -0.15, 
    "central_gravity" : 2.0, 
    "spring_constant" : 0.14,
    "avoid_overlap" : 0.5
    
}

d3_graph_settings = {
    "node_label_data_source" : "label", 
    "node_size_data_source" : "size", 
    "edge_size_data_source" : "size", 
    "edge_label_data_source" : "label", 
    "show_edge_label" : True,
    "edge_curvature" : 0.2,
    "use_collision_force" : True, 
    "collision_force_radius" : 60, 
    "collision_force_strength" : 0.8,
    "graph_height" : 800,
    "details_height" : 300, 
    
}

In [ ]:
gv.vis(nx_g, **vis_graph_settings)